In [1]:
var h = 0.00001
var a  = 3
var b = 4
var c = 5 + h 

var d = (a*b + c - (a*b + c - h) )/(h)

In [54]:
class Value {
  constructor(data, _children = [], _op = '', _label= '') {
    this.data = data;           // The actual value
    this._children = new Set(_children); // Set of child nodes in the computational graph
    this._backward = () => {};  // Function to propagate gradients
    this._op = _op;             // Operation used to create this value (add, mul, tanh, etc.)
    this.grad = 0;              // Gradient initialized to 0
    this._label = _label
  }

  // Addition operation
  add(operand) {
    if( !(operand instanceof Value)) {
        operand = new Value(operand)
    }
    // Create the output Value from the addition of two inputs
    const out = new Value(this.data + operand.data, [this, operand], '+');
    
    // Define the backward function for this addition
    out._backward = () => {
      this.grad += out.grad;    // Gradient for addition is passed equally to both operands
      operand.grad += out.grad;
    };
    
    return out;  // Return the result Value
  }

  // Multiplication operation
  mul(operand) {
    if( !(operand instanceof Value)) {
        operand = new Value(operand)
    }
    // Create the output Value from the multiplication of two inputs
    const out = new Value(this.data * operand.data, [this, operand], '*');
    
    // Define the backward function for this multiplication
    out._backward = () => {
      this.grad += operand.data * out.grad; // Gradient for multiplication w.r.t this
      operand.grad += this.data * out.grad; // Gradient for multiplication w.r.t operand
    };
    
    return out;  // Return the result Value
  }

  // Tanh activation function
    tanh() {
        const tanhData = Math.tanh(this.data);  // Compute tanh of the current value
        const out = new Value(tanhData, [this], 'tanh');

        // Define the backward function for tanh
        out._backward = () => {
          const tanhGrad = 1 - tanhData ** 2;  // Derivative of tanh(x) is 1 - tanh(x)^2
          this.grad += tanhGrad * out.grad;    // Chain rule: multiply by the gradient of the output
        };

        return out;  // Return the result Value
    }
    
    relU() {
        const out = new Value(Math.max(this.data, 0), [this], 'relU')
        
        out._backward = () => {
             this.grad += (this.data > 0) ? out.grad : 0;
        };
        return out
    }
    
    exp() {
        // Compute exponentiation
        const exp = Math.exp(this.data)
        const out = new Value(exp, [this], 'exp');

        // Define the backward function for this exponentiation
        out._backward = () => {
          this.grad += exp * out.grad; // Gradient w.r.t base
        };

        return out;  // Return the result Value
      }
    
    pow(operand) {
        // Compute exponentiation
        const out = new Value(Math.pow(this.data, operand), [this], '^');

        // Define the backward function for this exponentiation
        out._backward = () => {
          this.grad += operand * Math.pow(this.data, operand -1)*out.grad; // Gradient w.r.t base
        };

        return out;  // Return the result Value
    }
    
    sub(operand) {
        if( !(operand instanceof Value)) {
            operand = new Value(operand)
        }
        return this.add(operand.mul(-1))
    }
    
    div(operand) {
        return this.mul(operand.pow(-1))
    }
    
    backward() {
      // Topologically sort the nodes in the computational graph
      const topo = [];
      const visited = new Set();

      // Helper function to perform depth-first search (DFS) to order nodes
      const buildTopo = (v) => {
        if (!visited.has(v)) {
          visited.add(v);
          for (let child of v._children) {
            buildTopo(child);
          }
          topo.push(v);
        }
      };

      // Build the topological order starting from this node
      buildTopo(this);

      // Initialize the gradient of the output node (this node) to 1
      this.grad = 1;

      // Backpropagate through all nodes in reverse topological order
      for (let node of topo.reverse()) {
        node._backward();
      }
    }
}
    

In [60]:
class Neuron{
    constructor(num_in) {
        this.weights = Array.from({ length: num_in }, () => new Value(Math.random() * 2 - 1));
        this.bias = new Value(Math.random() * 2 - 1)
    }
    
    parameters(){
        let params_list = this.weights.concat([this.bias]);
        return params_list;
    }

    zero_grad() {
        this.weights.forEach(weight => {
            weight.grad = 0
        });

        this.bias.grad = 0
    }
    
    forward(x) {
        if (x.length !== this.weights.length) {
            throw new Error('Input vector length must match the number of neuron weights.');
        }
        let result = this.weights.reduce((sum, weight, i) => sum.add(weight.mul(x[i])), this.bias);
        return result;
    }
}

class Activation {
    constructor(act_func) {
        this.act_func = act_func
    }
    
    activate(neuron) {
        if(this.act_func == 'relU') {
            return neuron.relU()
        }
        else if (this.act_func == 'tanh') {
            return neuron.tanh()
        }
    }
}


class Layer {
    constructor(num_in, num_o, act_func) {
        this.num_in = num_in
        this.neurons = Array.from({ length: num_o }, () => new Neuron(num_in));
        this.act_func = Array.from({ length: num_o }, () => new Activation(act_func));
    }
    
    parameters() {
        return this.neurons.flatMap(neuron => neuron.parameters());
    }
    
    zero_grad() {
        this.neurons.forEach(neuron => {
            neuron.zero_grad()
        });
    }

    forward(x) {
        var logits = this.neurons.map(neuron => neuron.forward(x));
        const outs = Array.from({ length: this.act_func.length }, (_, i) => (this.act_func[i].activate(logits[i])));
        return outs
    }
}

class MLP {
    constructor(num_in, layer_sizes, activations) {
        this.layers = [];
        this.layers.push(new Layer(num_in, layer_sizes[0], activations[0]));

        for (let i = 1; i < layer_sizes.length -1; i++) {
            this.layers.push(new Layer(layer_sizes[i - 1], layer_sizes[i], activations[i]));
        }
        this.layers.push(new Layer(layer_sizes[layer_sizes.length - 2], 
                                   layer_sizes[layer_sizes.length - 1], 
                                   activations[layer_sizes.length - 1]));
    }
    
    parameters() {
        return this.layers.flatMap(layer => layer.parameters());
    }

    zero_grad()
    {
        this.layers.forEach(layer => {
            layer.zero_grad();
        });
    }
    
    forward(x) {
        let out = x;
        for (let layer of this.layers) {
            out = layer.forward(out);
        }
        if(out.length == 1) {
            return out[0]
        }
        return out;
    }
}

In [61]:
var x1 = new Value(2.0, [], '', 'x1');
var x2 = new Value(0.0, [], '', 'x2');

var w1 = new Value(-3, [], '', 'w1');
var w2 = new Value(1.0, [], '', 'w2');

var b = new Value(6.8813735870195432, [], '', 'b');

var x1w1 = x1.mul(w1)
x1w1._label = 'x1w1'
var x2w2 = x2.mul(w2)
x1w1._label = 'x2w2'

var x1w1x2w2 = x1w1.add(x2w2)
x1w1x2w2._label='x1w1x2w2'
var n = x1w1x2w2.add(b)
n._label = 'n'
// var o = n.tanh()
// o.backward()

"n"

In [62]:
var num = n.mul(2).exp().sub(1)
var denom = n.mul(2).exp().add(1)
var o = num.div(denom)
o.data

0.7071067811865477

In [63]:
o.backward()

In [65]:
var mlp = new MLP(3,  [4, 3, 2], ['relU', 'relU', 'tanh'])

In [66]:
mlp.forward([1, 2, 3])

[
  Value {
    data: 0.7503910556706261,
    _children: Set(1) {
      Value {
        data: 0.9738495159155278,
        _children: Set(2) { [Value], [Value] },
        _backward: [Function (anonymous)],
        _op: "+",
        grad: 0,
        _label: ""
      }
    },
    _backward: [Function (anonymous)],
    _op: "tanh",
    grad: 0,
    _label: ""
  },
  Value {
    data: 0.37352881353316053,
    _children: Set(1) {
      Value {
        data: 0.3925178507464653,
        _children: Set(2) { [Value], [Value] },
        _backward: [Function (anonymous)],
        _op: "+",
        grad: 0,
        _label: ""
      }
    },
    _backward: [Function (anonymous)],
    _op: "tanh",
    grad: 0,
    _label: ""
  }
]

# Generating Dataset

In [103]:
function targetFunction(x1, x2, x3) {
    return Math.sin(x1) + Math.pow(x2, 2) - Math.log(1 + Math.abs(x3));
}

// Function to generate synthetic data using the target function
function generateLearnableDataset(num_samples, num_features) {
    const data = [];
    const labels = [];

    for (let i = 0; i < num_samples; i++) {
        // Generate random input vector of size num_features
        let input = Array.from({ length: num_features }, () => Math.random() * 2 - 1);

        // Apply the target function to generate the label
        let labelValue = targetFunction(input[0], input[1], input[2]);
        let label = new Value(labelValue);  // Wrapping in the Value class

        // Convert the input to Value objects for the neural network
        let inputValues = input.map(val => new Value(val));

        data.push(inputValues);
        labels.push(label);
    }

    return { data, labels };
}
// var { data, labels } = generateLearnableDataset(30, 3)

var { data, labels } = {
    data: [
        [2, 3, -1],
        [3, -1, 0.5],
        [0.5, 1, 1],
        [1, 1, -1]
    ],
    labels: [1, -1, -1, 1]
};

In [114]:
var mlp = new MLP(3,  [4, 4,4, 1], ['relU', 'relU', 'relU', 'tanh'])

In [118]:
for(let i =0; i < 20; i++) {
    let outputs = Array.from({ length: data.length }, (_, i) => (mlp.forward(data[i])));
    let loss = outputs.reduce((sum, output, i) => sum.add(output.sub(labels[i]).pow(2)), new Value(0));
    
    
    mlp.zero_grad()
    loss.backward()
    var lr = 0.1
    var parameters = mlp.parameters()
    parameters.forEach(parameter => {
        parameter.data += -lr * parameter.grad;
    });
    if (i %1 == 0) {
        console.log(i, loss.data)
    }
}



0 0.01002753044938399
1 0.009689776788059618
2 0.009366109504461363
3 0.009061574798588812
4 0.008774448557797085
5 0.008497585647598595
6 0.008239568673247755
7 0.007990026562315476
8 0.007754769445866901
9 0.007530262133100016
10 0.007314846050081789
11 0.007112281120561932
12 0.006915342331163993
13 0.006729921513581668
14 0.006550865433857772
15 0.006379614934327272
16 0.00621686246865094
17 0.006058624329029394
18 0.005909471540912699
19 0.0057641572146894995


In [84]:
var a = [1, 3, 3, 4]